In [1]:

from pyspark.sql.functions import col, mean, stddev, sum, trim, row_number
from pyspark.sql import SparkSession, Window

# Initialize Spark Session with the same working configurations
spark = SparkSession.builder \
    .appName("Rearc Data Analysis") \
    .master("local[2]") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY")\
    .getOrCreate()

# Set up S3 credentials and Hadoop AWS configurations
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.endpoint", "s3.amazonaws.com")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("com.amazonaws.services.s3.enableV4", "true")
hadoop_conf.set("fs.s3a.aws.credentials.provider",
                "com.amazonaws.auth.InstanceProfileCredentialsProvider,com.amazonaws.auth.DefaultAWSCredentialsProviderChain")

print("SPARK SESSION CREATED SUCCESSFULLY")


25/03/14 17:16:58 WARN Utils: Your hostname, Abhijeets-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.7 instead (on interface en0)
25/03/14 17:16:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 17:16:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


SPARK SESSION CREATED SUCCESSFULLY


In [3]:

# AWS S3 Configuration
S3_BUCKET = "abh-de-rearc"
CSV_S3_PATH = f"s3a://{S3_BUCKET}/bls-data/pr.data.0.Current"
JSON_S3_PATH = f"s3a://{S3_BUCKET}/api-data/us_population.json"

# Load CSV file from S3 into PySpark DataFrame
print("Loading CSV data from S3...")
csv_df = spark.read.option("header", "false").option("delimiter", "\t").csv(CSV_S3_PATH)

# Rename columns properly
csv_df = csv_df.toDF("series_id", "year", "period", "value", "footnote_codes")

# Trim strings to remove unwanted spaces
csv_df = csv_df.withColumn("series_id", trim(col("series_id"))) \
               .withColumn("period", trim(col("period"))) \
               .withColumn("value", col("value").cast("float"))  # Ensure value is float

# Load JSON file from S3 into PySpark DataFrame
print("Loading JSON data from S3...")
json_df = spark.read.option("multiline", "true").json(JSON_S3_PATH)

# Extract relevant columns from JSON
population_df = json_df.selectExpr("explode(data) as data").select(
    col("data.Year").alias("year"),
    col("data.Population").alias("population")
)

# Convert population to integer
population_df = population_df.withColumn("population", col("population").cast("int"))


Loading CSV data from S3...


25/03/14 17:17:02 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://abh-de-rearc/bls-data/pr.data.0.Current.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.DataSou

Py4JJavaError: An error occurred while calling o38.csv.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(Dat

25/03/14 17:17:16 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:

# --- Task 1: Calculate Mean & Std Dev of Population (2013-2018) ---
pop_stats_df = population_df.filter((col("year") >= 2013) & (col("year") <= 2018))
pop_summary_df = pop_stats_df.select(mean("population").alias("mean_population"),
                                     stddev("population").alias("stddev_population"))

print("Population Statistics (2013-2018):")
pop_summary_df.show()


In [ ]:

# --- Task 2: Find Best Year for Each `series_id` (Max Sum of `value`) ---
agg_df = csv_df.groupBy("series_id", "year").agg(sum("value").alias("total_value"))

# Define a window specification to rank years by total_value per series_id
window_spec = Window.partitionBy("series_id").orderBy(col("total_value").desc())

# Assign ranks to each year and keep only the best (highest sum) year for each series_id
best_year_df = agg_df.withColumn("rank", row_number().over(window_spec)).filter(col("rank") == 1).drop("rank")

print("Best Year for Each Series ID:")
best_year_df.show()


In [ ]:

# --- Task 3: Report for `series_id = PRS30006032` and `period = Q01` ---
filtered_df = csv_df.filter((col("series_id") == "PRS30006032") & (col("period") == "Q01"))

# Join with population data
final_report_df = filtered_df.join(population_df, "year", "left").select(
    "series_id", "year", "period", "value", "population"
)

print("Final Report for `PRS30006032`, `Q01`:")
final_report_df.show()


In [ ]:

# Save all results as CSV in S3
output_path = f"s3a://{S3_BUCKET}/analytics-results/"
pop_summary_df.write.mode("overwrite").csv(output_path + "population_summary")
best_year_df.write.mode("overwrite").csv(output_path + "best_year_per_series")
final_report_df.write.mode("overwrite").csv(output_path + "final_report")

print(f"Results saved to: {output_path}")
